In [63]:
from openai import OpenAI
import anthropic
import google.generativeai as genai

import PIL.Image
import base64
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
chat_gpt = OpenAI()
claude = anthropic.Anthropic()
#genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
#gemini = genai.GenerativeModel('gemini-pro-vision')

In [54]:
def convert_png_to_jpeg(png_path, jpeg_path):
    img = PIL.Image.open(png_path)
    rgb_img = img.convert('RGB')
    rgb_img.save(jpeg_path, 'JPEG')
    
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def lanuage_models(system_text, user_text, image, max_tokens=1024):
    """
    Generates responses from multiple language models based on the given system text, user text, and image.

    Args:
        system_text (str): The system text to provide context for the conversation.
        user_text (str): The user text representing the user's input.
        image (str): The path to the image file.
        max_tokens (int, optional): The maximum number of tokens to generate in the response. Defaults to 1024.

    Returns:
        tuple: A tuple containing the responses from different language models.
            - chat_gpt_response (str): The response generated by the OpenAI GPT-4 Turbo model.
            - claude_response (str): The response generated by the Anthropic Claude model.
            - gemini_response (str): The response generated by the Google Gemini model.
    """
    convert_png_to_jpeg(image+'.png', image+'.jpeg')
    
    base64_image = encode_image(image+'.jpeg')
    pil_image = PIL.Image.open(image+'.jpeg')
    
    # openai
    response = chat_gpt.chat.completions.create(
        model="gpt-4-turbo",
        max_tokens=max_tokens,
        messages=[
            {"role": "system", "content": system_text},
            {"role": "user", "content": [
                {
                "type": "text",
                "text": user_text
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                }
                }]}],
            
    )
    
    chat_gpt_response = response.choices[0].message.content
    
    # anthropic
    response = claude.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=max_tokens,
        system=system_text,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": base64_image,
                        },
                    },
                    {
                        "type": "text",
                        "text": user_text
                    }
                ],
            }
        ],
    )
    
    claude_response = response.content[0].text
    
    # google
    #response = gemini.generate_content([system_text+ ' ' +user_text, pil_image], stream=True)
    gemini_response = 'X'#response.text
    
    return chat_gpt_response, claude_response, gemini_response
   

In [18]:
system_text = 'Stelle dir vor, dass du der beste deutschsprachige Medizinstudent bist. Du möchtest dein medizinisches Staatsexamen exzellent absolvieren! Nachfolgende werden dir Staatsexamina Fragen präsentiert. Gib als Lösung nur den richtigen Antwortbuchstaben aus!'

In [21]:
df_raw = pd.read_excel('data/questions.xlsx', sheet_name='AMBOSS - image')
df = df_raw[['overall number', 'total question ', 'case', 'correct answer']]
df.drop([50,51,52,53,54], inplace=True)
df['overall number'] = df['overall number'].astype(int)
df.rename(columns={'overall number': 'id', 'total question ': 'question', 'correct answer': 'correct_answer'}, inplace=True)
df.replace({'case': '-'}, float('NaN'), inplace=True)
df.replace('NaN', np.nan, inplace=True)

/var/folders/9n/dgw9qr6x53qb0klsv8fjtnc40000gn/T/ipykernel_46029/886374425.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop([50,51,52,53,54], inplace=True)
/var/folders/9n/dgw9qr6x53qb0klsv8fjtnc40000gn/T/ipykernel_46029/886374425.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overall number'] = df['overall number'].astype(int)
/var/folders/9n/dgw9qr6x53qb0klsv8fjtnc40000gn/T/ipykernel_46029/886374425.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

In [64]:
results = pd.DataFrame(columns=['id', 'correct_response', 'chat_gpt_response', 'claude_response', 'gemini_response'])

for id in tqdm(range(1, 3)):
    if df['case'][df['id'] == id].notna().all(): 
        user_text = df['case'][df['id'] == id].values[0] + df['question'][df['id'] == id].values[0]
    else:
        user_text = df['question'][df['id'] == id].values[0]
        
    chat_gpt_response, claude_response, gemini_response = lanuage_models(system_text, user_text, f'data/images/Amboss_SURG_images_{id:02}', max_tokens=1024)
    
    results = results._append({'id': id, 'correct_response': df['correct_answer'][df['id'] == id].values[0], 'chat_gpt_response': chat_gpt_response, 'claude_response': claude_response, 'gemini_response': gemini_response}, ignore_index=True)
    

100%|██████████| 2/2 [00:26<00:00, 13.03s/it]


In [65]:
results

,id,correct_response,chat_gpt_response,claude_response,gemini_response
0,1,B,B) arteriosklerotische pAVK,Die CT-Angiographie der unteren Extremitäten z...,X
1,2,E,B) linksseitiges Pulmonalisaneurysma,Die Computertomografie des Thorax zeigt eine i...,X


In [66]:
results.to_excel('results.xlsx', index=False)

In [ ]:
results = pd.DataFrame(columns=['id', 'correct_response' 'chat_gpt_response', 'claude_response', 'gemini_response'])

for id in df['id']:
    if df['case'][df['id'] == id].notna().all(): 
        user_text = df['case'][df['id'] == id].values[0] + df['question'][df['id'] == id].values[0]
    else:
        user_text = df['question'][df['id'] == id].values[0]
        
    chat_gpt_response, claude_response, gemini_response = lanuage_models(system_text, user_text, f'data/images/Amboss_SURG_images_{id:02}', max_tokens=1024)
    
    results.append({'id': id, 'correct_response': df['correct_response'][df['id'] == id].values[0], 'chat_gpt_response': chat_gpt_response, 'claude_response': claude_response, 'gemini_response': gemini_response}, ignore_index=True)
    